# COGS 108 - Final Project 

# Overview

People now intensively communicate on the social platform such as twitter, and it becomes the one of the most common ways for people to share, gather, and get influenced by information. We as a team wanted to explore how much different media may related to their follower's sentiment on twitter. We used text mining, AB test, and classification to carry out this project.


# Names

- Zirui Wang
- Yang Kuang
- Hengyu Liu

# Research Question

To what extent are mainstream medias' impact on their recent followers' speech on Twitter? 

# Background and Prior Work

#### References (include links):
- Garrett RK (2019) Social media’s contribution to political misperceptions in U.S. Presidential elections. PLoS ONE 14(3): e0213500. https://urldefense.com/v3/__https://doi.org/10.1371/journal.pone.0213500*5Cn__;JQ!!Mih3wA!QlQUFTK24Z4yWuHSjpnaVlfV8_Os401B_7fRaOqps2CZOe77QdZOeES4PhXe9ZE$ - Tajvidi, R., &amp; Karami, A. (2017, September 20). The effect of social media on firm performance. Retrieved October 24, 2020, from https://urldefense.com/v3/__https://www.sciencedirect.com/science/article/abs/pii/S0747563217305514?via=ihub*5Cn__;JQ!!Mih3wA!QlQUFTK24Z4yWuHSjpnaVlfV8_Os401B_7fRaOqps2CZOe77QdZOeES4Gfj2K2Q$ 
#### Background
- 1) People now are heavily rely on the technology and it becomes the one of the most common ways for people to gather information. However, there are so many different media sources. Different media sources have different stances and different ways to express or describe a contentious event. If one person only has one source of information, he or she very likely will believe that is the case. Later on, if he or she keeps receiving information only from this source, he or she starts to stay in the comfort zone/echo chambor. It becomes harder and harder for he/she to hear so many other voices addressed in other sources. Then, these people's views may closely related to the his/her main media source's view.
- 2) This topic becomes interesting to me because people often consider themselves as objective. However, their views may depend on their sources of information. Misinformation leads a lot of people to judge a contentious event from a narrow perspective.
- 3) This is important because this is a way that a nation can utilize its media to manipulate people's views/stances. Also, this is what happening in the world.
- 4) For real world example, media have great influence on U.S. Presidential elections. Certainly to some degree media becoming tools of political propaganda, but how effective they really are, and does this pattern also exist in other fields?
- 5) Our proposed research question is - To what extent are people's stances on contentious events on social media platforms related to their main media resources? We planned to randomly select 10K Twitter users and determine their main media source from its following list. However, it becomes so time consuming that it is impossible to access every user's following and follwer list. So, we changed our stretegy and tried to start with 10 mainstream media. As for each media, we collect its 1k recent followers. As for each follower, we collect its 200 recent tweets. In this way, we will ideally have 200K (200 * 1K) tweets for each media. Since there are cases like someone's tweets are unable to access or someone doesn't have at least 200 tweets, we didn't obtain 200K tweets for each media. They are roughly around 180K for each media. Now we have 10 groups of Twitter users' tweets according to each media. Going forward with our project, we will contact three different analyses on these tweets to see the impact of media on people's speech. Since our strategy changed a little bit, we rephrased our research question - To what extent are mainstream medias' impact on their recent followers' speech on Twitter? 

# Hypothesis


#### Hypothesis
We expect that some media will significantly influence its recent follower's views/stances/speech in a way that the users who followed different media will have different measurement of sentimental expression.

#### Explanation
There are so many media sources now. People usually choose their main media sources based on their preference in the beginning by following some media twitter accounts or subscribe to some newspaper. Later on, people tend to stay in their comfort zone/echo chamber and see posts/news which expressed in a way that they accepted/liked. When a controversial event happens, there may be various kinds of perspectives expressed by different media sources. Even there are cases like some specifc media are censoring specific topics they don't like. If people highly rely on their main media sources, it is possible that they only view that event through only the perspective of their main media source. 

# Setup

In [4]:
import gzip
import random
import numpy as np
import pandas as pd
import math
import json
import string
from sklearn.linear_model import LogisticRegression
import string
from nltk.corpus import stopwords
import nltk
from collections import defaultdict
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import  text
from scipy.sparse import hstack, csr_matrix
import tweepy as tw
import datetime
import json
import pickle
import re
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import  text
from sklearn.linear_model import LogisticRegression


# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name:
- Link to the dataset:
- Number of observations:

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

#### Dataset download

*Since these datasets are saved after pulling from the twitter api and too large to upload to Github, please download them here:* https://drive.google.com/file/d/1b94bDMC8Og_tUrG7AOOA4PqmJaEVqK1z/view?usp=sharing

#### Dataset 1
- Dataset Name: Media list
- Link to the dataset: media_ten
- Number of observations: 10
- Description: These are ten media that we started with. It contains the name of the media and their Twitter ID.

In [26]:
media = pd.read_csv('new_media.csv')
media_ten = media[:10]
media_ten

,news_media_twitter_id,news_media_name
0,nytimes,The New York Times
1,CNN,CNN
2,WSJ,The Wall Street Journal
3,washingtonpost,The Washington Post
4,EpochTimes,The Epoch Times
5,OANN,One America News
6,news_ntd,NTD News
7,FoxNews,Fox News
8,newsmax,Newsmax
9,nypost,New York Post


#### Dataset 2  
- Dataset Name: Recent 1000 followers for each media
- Link to the dataset: media_followers
- Number of observations: 10K (10 media * 1K followers)
- Description: This dataset contains 1K users' information for each media. Obtained previously and saved as "user_101000.txt".

In [27]:
# saved data file for "media_followers"
with open("user_101000.txt", "r") as fp:
    media_followers = json.load(fp)

In [49]:
for i in media_followers:
    print("Number of followers: " + str(len(media_followers[i])) + "\t Media: " + i)

Number of followers: 1000	 Media: nytimes
Number of followers: 1000	 Media: CNN
Number of followers: 1000	 Media: WSJ
Number of followers: 1000	 Media: washingtonpost
Number of followers: 1000	 Media: EpochTimes
Number of followers: 1000	 Media: OANN
Number of followers: 1000	 Media: news_ntd
Number of followers: 1000	 Media: FoxNews
Number of followers: 1000	 Media: newsmax
Number of followers: 1000	 Media: nypost


#### Dataset 3 
- Dataset Name: Recent 200 tweets for each follower for each media
- Link to the dataset: final
- Number of observations: 1723916
- Description: This dataset contains most recent 200 tweets for each follower in each media group. Ideally we will get 200 tweets for each follower, but some followers don't have at least 200 tweets posted. Noticed that OANN(One America News) has dramatically lower tweets that being collectable. 

In [24]:
# saved data file for "final"
with open("final1.txt", "r") as fp:
    final = json.load(fp)

In [48]:
#collected tweets from each media
sum = 0
for i in final:
    print("Number of tweets in total: " + str(len(final[i])) + " \t Media: " + i)
    sum += len(final[i])
    
print("Number of observations: " + str(sum))

Number of tweets in total: 169000 	 Media: nytimes
Number of tweets in total: 196000 	 Media: CNN
Number of tweets in total: 197000 	 Media: WSJ
Number of tweets in total: 194000 	 Media: washingtonpost
Number of tweets in total: 108000 	 Media: EpochTimes
Number of tweets in total: 83916 	 Media: OANN
Number of tweets in total: 186000 	 Media: news_ntd
Number of tweets in total: 194000 	 Media: FoxNews
Number of tweets in total: 197000 	 Media: newsmax
Number of tweets in total: 199000 	 Media: nypost
Number of observations: 1723916


# Data Cleaning/Processing (10%)

Describe your data cleaning steps here.

In [50]:
# read json saved data in to dataframe
def read_data(data):
    with open(data) as f:
        dict_1 = json.load(f)
    temp=[]
    for i in dict_1.keys():
        df_new=pd.DataFrame(dict_1[i],columns=['Post'])
        df_new['Media']=i
        temp.append(df_new)
    return pd.concat(temp)

In [51]:
#clean data, remove https, /n.
def clean_data(df):
    df['Post']=df['Post'].str.replace('\n','')
    df['Post']=df['Post'].apply(lambda x: x+' ')
    df['Post']=df['Post'].apply(lambda x: re.sub(r'http[s]?://.* ','',x))
    return df

#### Step 1: Read data
Originally data is in json format. The json dictionary has 10 keys, each of them is one media. Each key correspondes to a list where it contains all the tweets of that media. For every media, we have a dataframe. We merge all the dataframes together. In the end, we return the output. ('final1.txt' is larger than 100MB. We cannot upload it to GitHub.)

In [53]:
df=read_data('final1.txt')
df[:10]

,Post,Media
0,President-elect Joe Biden will nominate Pete B...,nytimes
1,President-elect Joe Biden intends to name Gina...,nytimes
2,Japanese prosecutors charged a man with arson ...,nytimes
3,A man was sentenced to four weeks in jail for ...,nytimes
4,Sweden has been an outlier in the global pande...,nytimes
5,Here's a look at some of the highlights worth ...,nytimes
6,Tom Cruise erupted at crew members on the set ...,nytimes
7,Delivery workers in South Korea say they feel ...,nytimes
8,Lawmakers in Mexico approved legislation Tuesd...,nytimes
9,MacKenzie Scott announced on Tuesday that she ...,nytimes


#### Step 2: Clean data
Firstly, we removed new line characters. Then, we removed the hyperlinks. At the end of the post, some of them has a space, but some of them doesn't. So we added a space at the end of each post so that we make sure every post has at least a space at the end of the post. In this way, it's easier to remove the hyperlinks which are usually placed by the followers in the end of the post by using regex.

In [57]:
df=clean_data(df)

#### Step 3
Sample 100000 tweets from each media's followers. Use all if less than 100000 tweets available. 


In [ ]:
random.seed(1)
medias = list(df['Media'].unique())
results = [0]*10
for i in range(10):
    media = medias[i]
    temp = df[df['Media']==media]
    results[i] = temp.sample(n=min(100000, temp.shape[0]), random_state=1, replace=False)
df = pd.concat(results)

#### Step 4: Text Cleasing/Processing

In [ ]:
# Tokenalize
df['Token'] = df['Post'].apply(word_tokenize)

In [ ]:
# Remove stop words and punctuation
punctuation = set(string.punctuation)
def remove_punt(l):
    return [i for i in l if not i in punctuation]
punctuation.update(['``', '’', "''", '”', '—', '“', "'s"])

stop_words = set(stopwords.words('english'))
def remove_sw(l):
    return [c for c in l if not c.lower() in punctuation and not c.lower() in stop_words]
df['Stop'] = df['Token'].apply(remove_sw)

In [ ]:
# PorterStemmer
from nltk.stem import PorterStemmer
ps = PorterStemmer()
df['Stem'] = df['Stop'].apply(lambda x: [ps.stem(y) for y in x])

#### Step 5: Text Sentiment

In [ ]:
# Sentimental analysis
tweets = defaultdict(pd.Series)
for i in df['Media'].unique():
    stem_tweets[i] = df[df['Media']==i]['Stem']
    tweets[i] = df[df['Media']==i]['Post'].apply(str)

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer 
analyser = SentimentIntensityAnalyzer()
scores = defaultdict(pd.Series)
for i in df['Media'].unique():
    scores[i] = tweets[i].apply(analyser.polarity_scores)

# Data Visualization (15%)

#### Step 1
Created a dataframe that contains the media and each post's compound score. Used the same method with read_data.

In [2]:
# font
font1 = {'family' : 'Times New Roman',
'weight' : 'normal',
'size'   : 15,
}

# read in saved sentimental data
def get_compound_score(d):
    return d['compound']
with open("media_sent_scores.txt", "r") as fp:
    scores_j = json.load(fp)
scores = defaultdict(pd.Series)
for i in scores_j:
    scores[i] = pd.Series(json.loads(scores_j[i]))
compound_scores = defaultdict(pd.Series)
for i in scores:
    compound_scores[i] = scores[i].apply(get_compound_score)

temp=[]
for i in compound_scores.keys():
    df_new=pd.DataFrame(compound_scores[i],columns=['Compound'])
    df_new['Media']=i
    temp.append(df_new)
df_c=pd.concat(temp)
df_c

FileNotFoundError: [Errno 2] No such file or directory: 'media_sent_scores.txt'

#### Step 2
Check each media compound score's summary. (count mean std min 25% 50% 75% max) and sort the media by the mean compound score from largest to smallest.

In [ ]:
df_ow=df_c.groupby('Media').describe()['Compound']
df_sort=df_ow.sort_values(by='mean',ascending= False)
df_sort

#### Step 3
Plot histgram for the compound score of each post of 10 media respectively. Used subplot to plot a 5x2 matritx plot. The graphs is also sorted by the mean compound score from largest to smallest, which corresponds to the previous step's sorting. From the graphs shown below, we can see that some media have concentrated netraul i.e. nypost, while others have relatively extremed distribution.

In [ ]:
plot_list=df_sort.index
plt.figure(figsize=(1000, 1000))
fig, axs = plt.subplots(2, 5,figsize=(40,20),sharey=True,sharex=False)
for i in range(10):
    axs[i//5, i%5].hist(compound_scores[plot_list[i]],color=sns.color_palette("tab10")[i])
    axs[i//5, i%5].set_ylabel('Frequency',font1)
    axs[i//5, i%5].set_xlabel('Compound_scores',font1)
    axs[i//5, i%5].set_title(plot_list[i],font1)
    axs[i//5, i%5].tick_params(labelsize=25,width=0.2)
fig.tight_layout()

<img src="graphs/hist.png">

#### Step 4
Used boxplot to plot a 5x2 matritx plot. The graphs here are one-to-one corresponds to the step 3 graphs in the same colors and positions.

In [ ]:
plt.figure(figsize=(50, 50))
fig, ax1 = plt.subplots(2,figsize=(20,20))
ax1[0] = sns.boxplot(ax=ax1[0],x="Media", y="Compound",data=df_c[df_c['Media'].isin(plot_list[:5])],order=plot_list[:5],
                     palette=sns.color_palette("tab10")[:5])
ax1[1] = sns.boxplot(ax=ax1[1],x="Media", y="Compound",data=df_c[df_c['Media'].isin(plot_list[5:])],order=plot_list[5:],
                     palette=sns.color_palette("tab10")[5:])
ax1[0].set_xlabel('Compound_score',font1)
ax1[0].set_ylabel('Media',font1)
ax1[1].set_xlabel('Compound_score',font1)
ax1[1].set_ylabel('Media',font1)

<img src="graphs/box.png">

# Data Analysis & Results (25%)

#### Step 1 AB Test
AB testing with α=0.05. We could use the compound score as a metric to check the influence of media on people since it is a continuous variable and also measures the sentiment of a sentence. And we can only test the two media which have the greatest difference in their mean score. According to the histogram, boxplot, and mean, we found that news_ntd and nypost have the greatest difference ( news_ntd-nypost = 0.083883 -0.097072=0.1809556239999914)in any pairs of media. So we can only run an AB test on these two with the largest differences. If there is no statistically significant difference between them. We can conclude that the media does not influence people's posts. Our H0: there are no differences between the compound score for the two media. H1: nypost has a lower compound score than news_ntd. We sample the data 500 times, randomly assign them to media to create data under H0. And we calculate the differences and compare them with our observed difference 0.1809556239999914. Then we calculate late the p-value which is the probability of seeing a difference of means at least as extreme as the observed, under the null hypothesis.
The p-value is 0.00<0.05 We reject the H0.

In [ ]:
n_repetitions = 500
df_t=df_c[df_c['Media'].isin(['news_ntd','nypost'])].reset_index(drop=True)
differences = []
for _ in range(n_repetitions):
    
    # shuffle the compound
    shuffled_compound = (
       df_t['Compound']
        .sample(replace=False, frac=1)
        .reset_index(drop=True)
    )
    
    # put them in a table
    shuffled = (
        df_t
        .assign(**{'Shuffled Compound': shuffled_compound})
    )
    
    # compute the group differences (test statistic!)
    group_means = (
        shuffled
        .groupby('Media')
        .mean()
        .loc[:, 'Shuffled Compound']
    )
    difference = group_means.diff().iloc[-1]
    
    # add it to the list of results
    differences.append(difference)

observed_difference = (
    df_t
    .groupby('Media')['Compound']
    .mean()
    .diff()
    .iloc[-1]
)

p_value=np.count_nonzero(differences <= observed_difference) / n_repetitions
p_value

#### Step 2 Classification
Since we obtain such significant difference in sentimential expression between 2 medias, we want to use classification as a predictive tool, featured on unigram of non-neutral sentiment post, try to predict whether this post come from one of these two medias.

<img src="graphs/non-n.jpg">

In [7]:
# "classification" contains all non-neutral sentiment post from nypost(New York Post) and news_ntd(NTD News).
c = pd.read_csv('classification.csv')

# get data from selected 2 medias
media_list = ['news_ntd','nypost']
d = c[c['Media'] == 'news_ntd']
e = c[c['Media'] == 'nypost']
f = pd.concat([d,e])

# process dataframe
tokenizer = nltk.RegexpTokenizer(r"\w+")
def get_Mids(m):
    return m == "news_ntd"
f['is_ntd'] = f['Media'].apply(get_Mids)
def unigrams(s):
    return list(tokenizer.tokenize(s))
f['Unigrams'] = f['Post'].apply(unigrams)


# shuffle and divide sets
shuffled = f.sample(frac=1, random_state=26)
train = shuffled[:100000]
validation = shuffled[100000:120000]
test = shuffled[120000:]

# training unigram
uC = defaultdict(int)
for i in train['Unigrams']:
    for j in i:
        uC[j]+=1
uniwords = sorted(list(uC.items()), key = lambda x:x[1], reverse = True)
# use 500 most frequent uniwords as one-hot coding features
uniword = [x[0] for x in uniwords[:500]]
uniID = dict(zip(uniword, range(len(uniword))))

# feature engeineering
def feature(datum):
    feat = [0]*len(uniword)
    uns = list(tokenizer.tokenize(datum))

    for un in uns:
        if un in uniword:
            feat[uniID[un]] += 1
    feat.append(1) #offset
    return feat

# training data
trainX = list(train['Post'].apply(feature))
trainy = list(train['is_ntd'])
valiX = list(validation['Post'].apply(feature))
valiy = list(validation['is_ntd'])
testX = list(test['Post'].apply(feature))
testy = list(test['is_ntd'])

solver = LogisticRegression(solver="liblinear",C=0.1)
solver.fit(trainX,trainy)

# prediction evaluation. Validation set result not included
predictions = solver.predict(testX)
accuracy = sum(predictions==testy)/len(testy)
accuracy

1.0

As for result, we got 100% accuracy on our classification model, meaning that there's clear difference about sentimental expression between two medias. This result confirms that certain media will strongly correlated to their followers in terms of emotions and characteristics of twittering. 

# Ethics & Privacy (15%)

- Acquire data from personal social media. Personal information might be involved.
- Generalize people’s viewpoint, might have misunderstandings or misinterpretations in comparing similarity.
- User's post or media may contain severely controversial topics.
- We didn't acquire the Twitter users consent on whether we can use/collect their tweets.
- We dropped the names and specific locaitons of any users to protect users' privacy. Even though we didn't show their names/id's, we exposed their tweets information/stances on different events.
- We used recent tweets from the Twitter. And recently the U.S. presidential election is still a hot topic. The liberals and conservatives have strongly opposing opinions. The news media are also exhibiting their stances, which may influence their followers in a greater impact than usual.
- Our results may expose Twitter users' political stances by analyzing their recent posts which may largely involve in the discussion of presidential election.
- Our data involves different people's tweets. Different people have different views on different events. Since our project and datasets may be published online, by look at our data, some people may feel offended by some of the tweets we collected.

# Conclusion & Discussion (15%)

By our EDA result, we found that selected 10 known medias have obvious difference sentimental distribution among their recent follower's tweets. This indicates that we can reasonably expect certain media will be followed by different user group, or their followers will be influenced by their recently followed media. Certain causation is not studied in this project, but we further carries out AB test to confirm this relationship. For two representitively selected medias, we found p-value=0 in AB test shows that such difference between medias are forceful/statistically significant. 
We also applied classification technic, examing the possibility to determine users' recently followed media by their recent post. Although its bi-outcome prediction, 100% accuracy testified such possibility that we can further applied on multiple medias.

Several concerns exist for this project, the most essential one is twitter API limitation. It has a strict limits on request betweens users relationship, users post, and other identity access. This project can be carried to boarder field with higher API access, such as followers' trend by given time, sample all followers instead of recent ones, generate users sentimental tread, etc. 
Due to time limits, we only do 10 media for EDA, and testifies 2 of them which expected to have distinctive results. Same precedure can be applied to larger variable pools, and by our EDA result variation, we can reasonably expect that such differences among medias on twitter are common phenomenon.
For collected data, we noticed that OANN(One America News) has dramatically lower tweets that being collectable. It's possibly because most of its new followers are inactive or private-post users, but reason behind that requires more study to find out. 

# Team Contributions

#### Yang Kuang
Yang worked on the data collection, data analyses, and conclusion.

#### Hengyu Liu
Hengyu worked on the data visulization and data analyses of the experiment.

#### Zirui Wang
Zirui Wang coordinated the team work. Zirui worked on the design, data collection, and report of the experiment.